In [1]:
import numpy as np 

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import decomposition
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVR # nu-SVR, implement from libsvm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn import preprocessing
# different regressor test

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier

import pandas as pd # process txt
from scipy.io import loadmat
from PIL import Image

from collections import defaultdict

# 40x85的输入，85x1024的矩阵，变换为40x1024的输出
import torch 
#import torch.nn as nn
#import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## **第0步**

In [2]:
folder = "G:/dataset/awa/"

standard_path = "G:/dataset/standard_split/AWA2/"
proposed_path = "G:/dataset/proposed_split/AWA2/" 

cls_to_idx = {}
with open(folder + "classes.txt", "r", encoding='utf-8') as f:
     for row in f.readlines():
         row = row.rstrip()
         idx, name = row.split()
         cls_to_idx[name] = int(idx)

sstrain, sstest = [], []
pstrain, pstest = [], []

with open(standard_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstrain.append(cls_to_idx[row])

with open(standard_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstest.append(cls_to_idx[row])

print("standard_split:", len(sstrain), len(sstest))
# transform List(str) -> List(int)

with open(proposed_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstrain.append(cls_to_idx[row])

with open(proposed_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstest.append(cls_to_idx[row])

print("proposed_split:", len(pstrain), len(pstest))

# Random Train & Test Class Split
X_class = list(range(1, 51))
train_class, test_class = train_test_split(X_class, test_size=0.2)

train_class, test_class = sstrain, sstest
train_class, test_class = pstrain, pstest

# labels
#yp = folder + "ResNet101/AwA2-labels.txt"
#y = np.loadtxt(yp, delimiter=" ", encoding='utf-8')


# visual features 2048 dimensions
#xp = folder + "ResNet101/AwA2-features.txt"
#x = np.loadtxt(xp, delimiter=" ", encoding='utf-8')

standard_split: 40 10
proposed_split: 40 10


In [3]:
test_class

[23, 50, 30, 24, 9, 34, 7, 47, 31, 41]

In [4]:
# semantic embedding of AwA2, 85 attibutes
seb = folder + "predicate-matrix-binary.txt" 
se = folder + "predicate-matrix-continuous.txt" 
# np.loadtxt(seb, delimiter=" ", encoding='utf-8'), binary works perfectly
# np.loadtxt(se, delimiter="  ", encoding='utf-8') # single, double, triple space exists
semat = np.zeros((50, 85))
with open(se, 'r', encoding='utf-8') as f:
    rows = f.readlines()
    cnt = 0
    for row in rows:
        row = row.strip()
        semat[cnt, :] = np.array(row.split(), dtype='float64')
        cnt = cnt + 1

semat.shape # unnormalize
semat = preprocessing.normalize(semat) 
# 标准化后和essay evaluation的一模一样

In [5]:
semat

array([[-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501],
       [ 0.12045618,  0.00426584,  0.        , ...,  0.17996306,
         0.0618086 ,  0.03495531],
       [ 0.26584459,  0.20652363,  0.        , ...,  0.05026822,
         0.04274552,  0.04915256],
       ...,
       [ 0.22516498,  0.15266022,  0.        , ...,  0.12733492,
         0.10009694,  0.01771   ],
       [ 0.19613947,  0.1966714 ,  0.        , ...,  0.01787277,
         0.06698743,  0.25883601],
       [ 0.03819588,  0.08046548,  0.10363715, ...,  0.01479997,
         0.05250999,  0.14194515]])

In [6]:
# 写成一个类方便其他人使用？
# combine x_n and y_n to a class
class awaRead:
    def __init__(self, p, train_split):
        """
        p: 数据集存放路径
        train_split: 给出训练集
        """
        X_class = list(range(1, 51))
        train_class = train_split
        test_class = list(filter(lambda i: i not in train_class, X_class))
        self.path = p
        # labels
        yp = self.path + "ResNet101/AwA2-labels.txt"
        y = np.loadtxt(yp, delimiter=" ", encoding='utf-8')
        # visual features 2048 dimensions
        xp = self.path + "ResNet101/AwA2-features.txt"
        x = np.loadtxt(xp, delimiter=" ", encoding='utf-8')
        i1 = np.isin(y, train_class)
        i2 = np.isin(y, test_class)
        self.X_train, self.X_test = x[i1], x[i2]
        self.y_train, self.y_test = y[i1], y[i2]
    def train_data(self):
        return self.X_train, self.y_train
    def test_data(self):
        return self.X_test, self.y_test
    def test(self):
        print(self.X_train.shape, self.y_train.shape)
        print(self.X_test.shape, self.y_test.shape)

#len(test_class), len(train_class) # (10, 40)
awaReader = awaRead(folder, train_class)
awaReader.test()

(29409, 2048) (29409,)
(7913, 2048) (7913,)


In [7]:
X_train, y_train = awaReader.train_data()
X_test, y_test = awaReader.test_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# hyper parameter
pca_d = 500

exemPCA = decomposition.PCA(n_components=pca_d) # 1024 => 500
exemPCA.fit(X_train)
X_train = exemPCA.transform(X_train)
X_test = exemPCA.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((29409, 500), (29409,), (7913, 500), (7913,))

## **第1步**

$$v_c = \frac {1} {|I_c|} \sum_{n \in I_c} M x_n$$

In [8]:
# group up PCA projections

exem_train_group = defaultdict(list)

for c in train_class:
    exem_train_group[c] = []
        
for x, y in zip(X_train, y_train):
    exem_train_group[y].append(x)


# Average
exem_train = {}
std_train = {}
k = 0

for item in exem_train_group.items():
    y, ary = item
    exem_train[y] = np.mean(ary, axis=0) # Key Sentence
    std_train[y] = np.std(ary, axis=0)

del exem_train_group

trai, tesi = [c-1 for c in train_class], [c-1 for c in test_class] # class to index, start from 0
a_c_train, a_c_test = semat[trai], semat[tesi]
v_c_train = [exem_train[i] for i in train_class] # dict uses classes to index
sd_train = [std_train[i] for i in train_class]
sd_train = np.mean(sd_train, axis=0)
# exem_train.keys(), train_class to test if they match
# e.g. extract the first dimension of exemplar to regression
# v_c_i = [vc[0] for vc in v_c_train]
# v_c_train[:3], v_c_i[:3] # dimension 0 data
# len(v_c_train), len(v_c_train[0]) 40 1024

In [9]:
print(np.array(a_c_train).shape, np.array(v_c_train).shape, np.array(sd_train).shape)
###
x, y = torch.tensor(np.array(a_c_train), dtype=torch.float), torch.tensor(np.array(v_c_train), dtype=torch.float)

(40, 85) (40, 500) (500,)


## 第2步 
### Training

用MLP训练，


X为语义表示，是固定的a_c_train矩阵，维度为40x85转换为40x1024

$$\psi (a_c) \approx v_c$$

In [10]:
# Torch Training

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 40, 85, 300, pca_d
# Create random Tensors to hold inputs and outputs
#x, y = torch.randn(N, D_in), torch.randn(N, D_out)
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    #torch.nn.Dropout(p=0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(7000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 350 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

# 9999 2016.8387451171875

99 12925.2529296875
449 10871.498046875
799 8397.27734375
1149 6373.3681640625
1499 4843.3095703125
1849 3697.82275390625
2199 2828.2646484375
2549 2151.72998046875
2899 1620.2830810546875
3249 1205.9376220703125
3599 884.9330444335938
3949 639.417236328125
4299 453.34844970703125
4649 313.6085205078125
4999 209.99050903320312
5349 135.51856994628906
5699 84.28010559082031
6049 50.39766311645508
6399 28.86829948425293
6749 15.65655517578125


In [11]:
#test_x = torch.randn(10, 85)
model.eval()
test_y = model(torch.tensor(a_c_test, dtype=torch.float))
test_y.shape
v_c_test = test_y.detach().numpy()
# v_c_test = np.zeros((10, 1024)) # 提前定义好exemplar矩阵

print(v_c_test.shape, len(v_c_train), len(v_c_train[0]))


exem_X, exem_y = [], []

for i, c in enumerate(test_class):
    exem_X.append(v_c_test[i])
    exem_y.append(c)

# Add this part to become GZSL
#for i, c in enumerate(train_class):
#    exem_X.append(v_c_train[i])
#    exem_y.append(c)

#X_test.shape
# exem_X.shape, exem

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(exem_X, exem_y)
print("1NN:{} ".format(neigh.score(X_test, y_test)))

sneigh = KNeighborsClassifier(n_neighbors=1, metric='seuclidean', 
                    metric_params={'V':sd_train})
sneigh.fit(exem_X, exem_y)
print("1NNs:{}".format(sneigh.score(X_test, y_test)))

(10, 500) 40 500
1NN:0.6258056362947049 
1NNs:0.5803108808290155


### Example Log of Training

```
99 13051.6328125
449 11867.4072265625
799 10301.38671875
1149 9025.2138671875
1499 8009.3193359375
1849 7143.935546875
2199 6734.91357421875
2549 6102.67724609375
2899 5725.56787109375
3249 5473.65673828125
3599 5151.4560546875
3949 4954.791015625
4299 4683.5986328125
4649 4459.27197265625
4999 4372.97705078125
5349 4017.916748046875
5699 4114.7744140625
6049 3887.33935546875
6399 3573.56396484375
6749 3627.4140625

1NN:0.6766081132313914 
1NNs:0.6683937823834197
```

## 第3步 alter
### Predicting

对于a_c_test(10x85)，使用MLP预测得到10x1024的矩阵，这就是我们要求的未见类的视觉特征代表。

$\psi (a_u) = v_u$

## 第3步 original
### Predicting

对于a_c_test(10x85)，依次使用回归器预测出10x1的一个维度的结果，填入结果的一个列中。循环了1024次后得到10x1024的矩阵，这就是我们要求的未见类的视觉特征代表。

$\psi (a_u) = v_u$

## 第4步
### Testing
突然发现如果是做零样本学习，exem_X就不该加入可见类

$\hat y = arg \min dis_{NN} (\psi(M x), v_u) $

最近邻分类器，X (10, 1024) y (10, 1)

注意v_c_test的顺序，由a_c_test决定，所以根据tesi来分配y


In [12]:
test_class

[23, 50, 30, 24, 9, 34, 7, 47, 31, 41]

### Original

```
regress_group = []
k = 0
for j in range(pca_d):
    X = a_c_train # 
    y = [vc[j] for vc in v_c_train]
    regressor = NuSVR(C=2)
    regressor.fit(X, y)
    regress_group.append(regressor)
    #print(len(X), len(X[j]), len(y))
    #regress_group[j].fit(X, y)

# validate
# regress_group[10].predict([a_c_train[3]]), v_c_train[3][10]

v_c_test = np.zeros((10, pca_d)) # 提前定义好exemplar矩阵
# 对每一个维度进行预测
for j in range(pca_d):
    v_c_test[:, j] =  regress_group[j].predict(a_c_test) # 10 dimension , assign to column

v_c_test.shape, len(v_c_train), len(v_c_train[0])


exem_X, exem_y = [], []

for i, c in enumerate(test_class):
    exem_X.append(v_c_test[i])
    exem_y.append(c)

# Add this part to become GZSL
#for i, c in enumerate(train_class):
#    exem_X.append(v_c_train[i])
#    exem_y.append(c)

#X_test.shape
# exem_X.shape, exem

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(exem_X, exem_y)
print("1NN:{} ".format(neigh.score(X_test, y_test)))

sneigh = KNeighborsClassifier(n_neighbors=1, metric='seuclidean', 
                    metric_params={'V':sd_train})
sneigh.fit(exem_X, exem_y)
print("1NNs:{}".format(sneigh.score(X_test, y_test)))
```

MLP用于最后的预测


```python

mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=30, alpha=1e-4,
                    solver='adam', verbose=10, random_state=1,
                    learning_rate_init=.01)
mlp.fit(exem_X, exem_y)
mlp.score(X_test, y_test)
```

In [13]:
accuracy = neigh.score(X_test, y_test)
#neigh.predict(X_test[:2])

In [14]:
#X_test.shape, y_test.shape, np.array(exem_X).shape, np.array(exem_y).shape

In [15]:
## Save Exemplars and visual features of test class
#save_path = "./awa2_exem/"
#np.save(save_path + "exem_test.npy", np.array(exem_X))
#np.save(save_path + "X_test.npy", X_test)
#np.savetxt(save_path + "y_test.txt", np.array(y_test, dtype=int), fmt='%s', encoding='utf-8')

### MLP改进

MLP用于psi函数的学习，并进行视觉特征代表的预测

- 4.17 proposed split:
    0.645899153292051 ZSL